<a href="https://www.kaggle.com/code/theongy/getting-started-maneuver-detection?scriptVersionId=120421670" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
"""Command line to load the data (valid only fr this notebook)"""
import sys
package_paths = [
    '../input/hackathon2023/',
]

for pth in package_paths:
    sys.path.append(pth)
!cp -r '../input/maneuver-detection/' '/kaggle/working'

# Load the dataset

In [2]:
from ManeuverDetectionDataset import ManeuverDetectionDataset
dataset= ManeuverDetectionDataset("/kaggle/working/maneuver-detection/TEST_FILE_PUBLIC.json")



**********TRAIN DATASET *********
Validation/Train ratio: 0.1
Samples filtered? NO
Samples evenly spaced? False
path: /kaggle/working/maneuver-detection/TEST_FILE_PUBLIC.json
loading dataset. Ready in a minute!
TRAIN Dataset loaded. Size: 1403
